In [1]:
import tequila as tq
import warnings
import numpy as np
warnings.filterwarnings("ignore")

/home/liamj/chemical-graph/tequila-beta/lib/python3.12/site-packages/tequila/simulators/simulator_api.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# $H_4$ Ground State Simulations

In [2]:
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

In [3]:
# Lets look at the Tapered Binary transformation
mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="TaperedBinary").use_native_orbitals()

In [4]:
# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)

HF  -1.8291374124430173
FCI  -1.9961503255188089


Method 1: SPA

In [6]:
SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H = mol.make_hamiltonian()

# IS THIS G1?
# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U = U0 + SPA + U0.dagger()
U_cirq = tq.compile(U, backend="cirq")
res = tq.minimize(tq.ExpectationValue(H=H,U=U),silent=True)
print(" ----------------------- SPA (G1) ----------------------- ")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
# print(U)
print(U_cirq.circuit)
print(f"Measure Hamiltonian H = {H}\n")
try:
    U.export_to(filename="h4_ground_state_tapered_G1.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

 ----------------------- SPA (G1) ----------------------- 
Energy: -1.9562546212337821
difference from fci: 39.89570428502675 meh?
                                                                                                                                                                                                                                                                                                                        ┌──────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌──────┐
0: ───X^0.5────@────────────

Method 2: Classical optimization

In [7]:
# THIS JUST CLASSICALLY OPTIMIZES THE ORBITALS?
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=SPA, initial_guess=guess, silent=True).molecule
H_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=SPA), silent=True)
SPA_cirq = tq.compile(SPA, backend="cirq")
print("classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
print(SPA_cirq.circuit)
print(f"Measure Hamiltonian H = {H_opt}\n")
try:
    SPA.export_to(filename="h4_ground_state_tapered_G1_classically_optimized_orbitals.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

classically optimized orbitals
Energy: -1.979887884543346
difference from fci: 16.26244097546281 meh?

0: ───X───────────────────────────────────────────────────X───────────────────────────────────────────────────@───
                                                          │                                                   │
1: ───────────────────────────────────────────────────────┼───────────────────────────────────────────────────X───
                                                          │
2: ───Y^(0.318309886183791*f((((0, 1), 'D', None),))_0)───@───────────────────────────────────────────────────@───
                                                                                                              │
3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────X───

4: ───X───────────────────────────────────────────────────Y^(0.318309886183791*f((((2, 3), 'D', None),))_1)───@───
                                     

Method 3: Orbital Correlator (two graphs)

In [ ]:
# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_p = SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H,U=U_p), silent=True)
U_p_cirq = tq.compile(U_p, backend="cirq")
print("SPA + Correlator (G2)")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
print(U_p_cirq.circuit)
print(f"Measure Hamiltonian H = {H}\n")
try:
    U_p.export_to(filename="h4_ground_state_tapered_G2.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

Method 4: Orbital Correlator + Optimized Hamiltonian

In [ ]:
# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_plus = SPA
U_plus += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_plus += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_plus += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=U_plus),silent=True)
U_plus_cirq = tq.compile(U_plus, backend="cirq")
print("SPA + Correlator (G2) + classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
print(U_plus_cirq.circuit)
print(f"Measure Hamiltonian H = {H_opt}\n")
# try:
#     U.export_to(filename="overlap_circuit.pdf")
# except Exception as E:
#     print("no qpic and/or latex installed ... sorry")

In [ ]:
# Now everything without tapering to compare:
print("\n -----------NO TAPERING----------- \n ")

mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="JordanWigner").use_native_orbitals()

# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)

SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H = mol.make_hamiltonian()

# IS THIS G1?
# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U = U0 + SPA + U0.dagger()
U_cirq = tq.compile(U, backend="cirq")
res = tq.minimize(tq.ExpectationValue(H=H,U=U),silent=True)
print("SPA (G1)")
print(f"Energy: {res.energy}")
print(U)
print(U_cirq.circuit)
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
try:
    U.export_to(filename="h4_ground_state_JW_G1.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

# THIS JUST CLASSICALLY OPTIMIZES THE ORBITALS?
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=SPA, initial_guess=guess, silent=True).molecule
H_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=SPA), silent=True)
print("classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")



# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_p = SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H,U=U_p), silent=True)
print("SPA + Correlator (G2)")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")


# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_plus = SPA
U_plus += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_plus += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_plus += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_opt,U=U_plus),silent=True)
print("SPA + Correlator (G2) + classically optimized orbitals")
print(f"Energy: {res.energy}")
print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")